# Aula 06.Ex02 - Malhas e Texturas - Mapeamento de Texturas

### Primeiro, vamos importar as bibliotecas necessárias.

In [3]:
import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shader_s import Shader

### Inicializando janela

In [5]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(700, 700, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### Constroi e compila os shaders. Também "linka" eles ao programa

#### Novidade aqui: modularização dessa parte do código --- temos agora uma classe e arquivos próprios para os shaders (vs e fs)
Créditos: https://learnopengl.com

In [7]:
ourShader = Shader("vertex_shader.vs", "fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()

### Preparando dados para enviar a GPU

Até aqui, compilamos nossos Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront (.obj).

Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file

In [10]:
glEnable(GL_TEXTURE_2D)
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)


global vertices_list
vertices_list = []    
global textures_coord_list
textures_coord_list = []


def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model


def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)



'''
É possível encontrar, na Internet, modelos .obj cujas faces não sejam triângulos. Nesses casos, precisamos gerar triângulos a partir dos vértices da face.
A função abaixo retorna a sequência de vértices que permite isso. Créditos: Hélio Nogueira Cardoso e Danielle Modesti (SCC0650 - 2024/2).
'''
def circular_sliding_window_of_three(arr):
    if len(arr) == 3:
        return arr
    circular_arr = arr + [arr[0]]
    result = []
    for i in range(len(circular_arr) - 2):
        result.extend(circular_arr[i:i+3])
    return result


def load_obj_and_texture(objFile, texturesList):
    modelo = load_model_from_file(objFile)
    
    ### inserindo vertices do modelo no vetor de vertices
    verticeInicial = len(vertices_list)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, len(vertices_list)))
    faces_visited = []
    for face in modelo['faces']:
        if face[2] not in faces_visited:
            faces_visited.append(face[2])
        for vertice_id in circular_sliding_window_of_three(face[0]):
            vertices_list.append(modelo['vertices'][vertice_id - 1])
        for texture_id in circular_sliding_window_of_three(face[1]):
            textures_coord_list.append(modelo['texture'][texture_id - 1])
        
    verticeFinal = len(vertices_list)
    print('Processando modelo {}. Vertice final: {}'.format(objFile, len(vertices_list)))
    
    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    if texturesList:
        for id in range(len(texturesList)):
            load_texture_from_file(id,texturesList[id])
    
    return verticeInicial, verticeFinal - verticeInicial

In [11]:
load_model_from_file('caixa/caixa.obj') #apenas para ilustrar como é o resultado da função

{'vertices': [['-1', '-1', '1'],
  ['-1', '-1', '-1'],
  ['1', '-1', '-1'],
  ['1', '-1', '1'],
  ['-1', '1', '1'],
  ['-1', '1', '-1'],
  ['1', '1', '-1'],
  ['1', '1', '1']],
 'texture': [['1.000000', '0.000000'],
  ['0.000000', '1.000000'],
  ['0.000000', '0.000000'],
  ['1.000000', '0.000000'],
  ['0.000000', '1.000000'],
  ['0.000000', '0.000000'],
  ['1.000000', '0.000000'],
  ['0.000000', '1.000000'],
  ['0.000000', '0.000000'],
  ['1.000000', '0.000000'],
  ['0.000000', '1.000000'],
  ['0.000000', '0.000000'],
  ['1.000000', '0.000000'],
  ['0.000000', '0.000000'],
  ['0.000000', '1.000000'],
  ['1.000000', '1.000000'],
  ['1.000000', '1.000000'],
  ['1.000000', '1.000000'],
  ['1.000000', '1.000000'],
  ['1.000000', '1.000000']],
 'faces': [([6, 1, 5], [1, 2, 3], None),
  ([7, 2, 6], [4, 5, 6], None),
  ([8, 3, 7], [7, 8, 9], None),
  ([5, 4, 8], [10, 11, 12], None),
  ([2, 4, 1], [13, 11, 14], None),
  ([7, 5, 8], [4, 15, 12], None),
  ([6, 2, 1], [1, 16, 2], None),
  ([7, 3,

### Vamos carregar cada modelo e definir funções para desenhá-los

#### Novidade aqui: note que a função "desenha_caixa" é diferente agora!

In [13]:
# carrega caixa (modelo e texturas)
verticeInicial, quantosVertices = load_obj_and_texture('caixa/caixa.obj', ['caixa/caixa.jpg', 'caixa/matrix.jpg', 'caixa/tijolos.jpg'])


def desenha_caixa(tx, ty, tz, idTextura, sequencia):


    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)

    

    mat_translate       = np.array([1, 0.0, 0.0, tx, 
                                    0.0, 1, 0.0, ty, 
                                    0.0, 0.0, 1, tz, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)  


    mat_translate_inv       = np.array([1, 0.0, 0.0, -tx, 
                                    0.0, 1, 0.0, -ty, 
                                    0.0, 0.0, 1, -tz, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)  


    mat_scale       = np.array([    0.1, 0.0, 0.0, 0.0, 
                                    0.0, 0.1, 0.0, 0.0, 
                                    0.0, 0.0, 0.1, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)    
   


    # A sequencia abaixo ...
    if sequencia == 0:
        mat_transform = mat_translate_inv #primeira transformacao
    
        mat_transform = multiplica_matriz(mat_scale,mat_transform) #segunda...
    
        mat_transform = multiplica_matriz(mat_rotation_x,mat_transform)
        mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)
        mat_transform = multiplica_matriz(mat_rotation_z,mat_transform)
        
        mat_transform = multiplica_matriz(mat_translate,mat_transform) #ultima


    # ... tem o mesmo efeito dessa
    else:
        mat_transform = mat_translate #ultima
    
        mat_transform = multiplica_matriz(mat_transform, mat_scale) # penultima...
    
        mat_transform = multiplica_matriz(mat_transform, mat_rotation_z)
        mat_transform = multiplica_matriz(mat_transform, mat_rotation_y)
        mat_transform = multiplica_matriz(mat_transform, mat_rotation_x)
        
        mat_transform = multiplica_matriz(mat_transform, mat_translate_inv) #primeira transformacao
    

    loc_mat_transform = glGetUniformLocation(program, "mat_transform")
    glUniformMatrix4fv(loc_mat_transform, 1, GL_TRUE, mat_transform) 
           
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, idTextura)
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, verticeInicial, quantosVertices) ## renderizando

Processando modelo caixa/caixa.obj. Vertice inicial: 0
Processando modelo caixa/caixa.obj. Vertice final: 36


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar dois slots (buffers): um para os vértices e outro para as texturas.

In [15]:
buffer_VBO = glGenBuffers(2)

### Enviando coordenadas de vértices para a GPU

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [17]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

### Enviando coordenadas de textura para a GPU

In [19]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Nesse momento, nós exibimos a janela!


In [21]:
glfw.show_window(window)

### Loop principal da janela.

In [23]:

glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c


angulo_rotacao= 0 
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    angulo_rotacao -= 0.01 # modifica o angulo de rotacao em cada iteracao
    cos_d = math.cos(angulo_rotacao)
    sin_d = math.sin(angulo_rotacao)
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)

    desenha_caixa(0.5,0.5,0,0,1)
    desenha_caixa(0,0.1,0,1,0)
    desenha_caixa(-0.5,-0.5,0,2,1)


    glfw.swap_buffers(window)

glfw.terminate()